In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List
from moseq2_nlp.data import get_transition_representations_n, sample_markov_chain
from moseq2_nlp.train import train_regressor, train_svm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pdb
import os
from tqdm import tqdm
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

# Synthetic data

What are you doing here....?

## Synthesize data

In [2]:
data_dir = '/cs/labs/mornitzan/ricci/data/abraira'
data_name = '2020-11-10_Celsr3_R774H'
model_file = os.path.join(data_dir, data_name, 'robust_septrans_model_1000.p')
index_file = os.path.join(data_dir, data_name, 'gender-genotype-index.yaml')

# Get nth order transitions (usages, transitions, 3grams, etc.)
n=1
group_transition_arrays = get_transition_representations_n(model_file, index_file, n)

Loading raw data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:03<00:00, 26.76it/s]


Getting 1-grams for F_+/+.


  0%|                                                                                                                                                                                                       | 0/27345 [00:00<?, ?it/s]/cs/labs/mornitzan/ricci/projects/moseq2-nlp/moseq2_nlp/data.py:170: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  group_transition_arrays[group][ind] += 1
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27345/27345 [00:00<00:00, 325405.98it/s]


Getting 1-grams for F_RH/RH.


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49434/49434 [00:00<00:00, 328559.72it/s]


Getting 1-grams for M_RH/RH.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14749/14749 [00:00<00:00, 320006.77it/s]


Getting 1-grams for M_+/+.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22954/22954 [00:00<00:00, 325317.39it/s]


In [3]:
num_animals_per_group = 25
num_syllables_range = [10000,15000]
all_synthesized_data = []
labels = []

for l, (group, tmx) in enumerate(group_transition_arrays.items()):
    print(f'Synthesizing {group}.')
    for _ in tqdm(range(num_animals_per_group)):
        num_syllables = np.random.randint(num_syllables_range[0], num_syllables_range[1])
        all_synthesized_data.append(sample_markov_chain(tmx,num_syllables))
        labels.append(l)
        
documents = [TaggedDocument(sent, [i]) for i, sent in enumerate(all_synthesized_data)]

Synthesizing F_+/+.


  0%|                                                                                                                                                                                                          | 0/25 [00:00<?, ?it/s]/cs/labs/mornitzan/ricci/projects/moseq2-nlp/moseq2_nlp/data.py:197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  probs = np.squeeze(tmx[ind])
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  6.73it/s]


Synthesizing F_RH/RH.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  6.58it/s]


Synthesizing M_+/RH.


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  6.68it/s]


Synthesizing M_RH/RH.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  6.73it/s]


Synthesizing M_+/+.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  6.63it/s]


## Train on synthetic data

In [5]:
dim = 300 #max_syllable # Dimension of embedding space
window = 1 # Window size for context (this is left and right, so total size is 2*window)

# Initialize and train two models on the documents. Embeddings will be averaged, which is standard. 
# Note: min_count = <min_count> omits words with usages less than <min_count>
print('Training dm=1')
model1 = Doc2Vec(documents, dm=1, epochs=50, vector_size=dim, window=window, min_count=1, workers=1)
print('Training dm=0')
model2 = Doc2Vec(documents, dm=0, epochs=50, vector_size=dim, window=window, min_count=1, workers=1)
print('Done')

Training dm=1
Training dm=0
Done


In [6]:
# Infer embeddings per document per model and then average. 
E1 = [model1.infer_vector(sent) for sent in all_synthesized_data]
E2 = [model2.infer_vector(sent) for sent in all_synthesized_data]
E = [.5 * (em1 + em2) for (em1, em2) in zip(E1, E2)]

## Classify

In [10]:
classifier = 'logistic_regression'
scoring = 'accuracy'
K = 1
penalty = 'l2'
num_c = 11
seed = 0

print('Training classifier')
if classifier == 'logistic_regression':
    best_C, best_score = train_regressor(E, labels, K, scoring, penalty, num_c, seed)
elif classifier == 'svm':
    best_C, best_score = train_svm(E, labels, kernel, K, scoring, penalty, num_c, seed)
    
print(best_C, best_score)

Training classifier
0.1 1.0
